In [1]:
import pandas as pd
from textblob import TextBlob
import math
from sklearn.preprocessing import MinMaxScaler
from matplotlib.pyplot import MultipleLocator
import matplotlib.pyplot as plt
import numpy as np


In [2]:
df_news = pd.read_csv("data/Bai_news_headlines.csv")
df_news.date = pd.to_datetime(df_news.date)

In [4]:
h_matrix_file = open('seanmf_results/4H.txt','r',encoding='utf-8')

In [5]:
df_prop = pd.DataFrame(columns = ['date','topic1','topic2','topic3','topic4'])
t1_l,t2_l,t3_l,t4_l,t5_l,t6_l= [],[],[],[],[],[]

for line in h_matrix_file:
    values = str(line).split(' ')
    values = [float(i) for i in values]
    t1_l.append(values[0]/(sum(values))) ## The first entry is the word
    t2_l.append(values[1]/(sum(values)))
    t3_l.append(values[2]/(sum(values)))
    t4_l.append(values[3]/(sum(values)))


h_matrix_file.close()
df_prop['topic1'] = t1_l
df_prop['topic2'] = t2_l
df_prop['topic3'] = t3_l
df_prop['topic4'] = t4_l


In [6]:
df_news.shape

(28220, 2)

In [7]:
df_prop.shape

(28220, 5)

In [8]:
df_prop['date'] = df_news.date

In [11]:

df_grouped_prop = df_prop.groupby(['date']).mean()
df_grouped_prop.reset_index(inplace=True)


In [21]:
df_grouped_prop.columns

Index(['date', 'topic1', 'topic2', 'topic3', 'topic4'], dtype='object')

In [13]:
df_sentiment = pd.read_csv("Bai_news_headlines_sentiment_indicators.csv")
df_sentiment.date = pd.to_datetime(df_sentiment.date)
df_sentiment['normalized_polarity']= MinMaxScaler().fit_transform(np.array(df_sentiment['cumulated_polarity']).reshape(-1, 1))


In [37]:
df_sentiment.shape

(2583, 4)

In [16]:
df_price = pd.read_csv("data/Bai_origin_oil.csv")
df_price.date = pd.to_datetime(df_price.date)
df_price['normalized_price']= MinMaxScaler().fit_transform(np.array(df_price['price']).reshape(-1, 1))


In [35]:
df_price.shape

(2008, 3)

In [36]:
df_grouped_prop.shape

(2583, 5)

In [ ]:
df_polarity_price=pd.merge(df_sentiment,df_price,on=["date"])[["date","normalized_polarity","normalized_price"]]


In [34]:
df_polarity_price=pd.merge(df_sentiment,df_price,on=["date"])[["date","normalized_polarity","normalized_price"]]
df_polarity_price

,date,normalized_polarity,normalized_price
0,2011-03-29,0.820735,0.896216
1,2011-03-30,0.789064,0.890023
2,2011-03-31,0.783535,0.917431
3,2011-04-01,0.806864,0.933028
4,2011-04-04,0.923509,0.939794
...,...,...,...
1988,2019-03-18,0.731557,0.377294
1989,2019-03-19,0.776877,0.377638
1990,2019-03-20,0.761152,0.389106
1991,2019-03-21,0.780255,0.387500


In [38]:
df_final=pd.merge(df_polarity_price,df_grouped_prop,on=["date"])

In [39]:
df_final

,date,normalized_polarity,normalized_price,topic1,topic2,topic3,topic4
0,2011-03-29,0.820735,0.896216,0.322602,0.633046,0.001900,0.042452
1,2011-03-30,0.789064,0.890023,0.389422,0.542997,0.034930,0.032651
2,2011-03-31,0.783535,0.917431,0.373578,0.593646,0.011477,0.021299
3,2011-04-01,0.806864,0.933028,0.360771,0.539566,0.070864,0.028800
4,2011-04-04,0.923509,0.939794,0.341547,0.561921,0.048073,0.048459
...,...,...,...,...,...,...,...
1988,2019-03-18,0.731557,0.377294,0.495080,0.353325,0.077713,0.073882
1989,2019-03-19,0.776877,0.377638,0.439608,0.414550,0.088073,0.057769
1990,2019-03-20,0.761152,0.389106,0.563433,0.289652,0.081781,0.065134
1991,2019-03-21,0.780255,0.387500,0.491381,0.346630,0.094365,0.067624


In [51]:
df_final.to_csv("integrated_series.csv",index=False)